<a href="https://colab.research.google.com/github/AIFFEL-GN-2nd/TotochTeam3/blob/main/ex3_albumentations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi
!python --version

Thu Oct  7 16:53:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 1. 데이터셋 불러오기

In [ ]:
# 데이터셋의 압축을 풀어보자.

!pip install gdown
!gdown https://drive.google.com/uc?id=1RBVuPoOFTTSGioAOsUzy76wPSFg3hw5J
!gdown https://drive.google.com/uc?id=1AIIsrHpftkGEw0-8iKoLLrPoRCNWHq5R
!unzip -q ./train.zip
!unzip -q ./test1.zip

Downloading...
From: https://drive.google.com/uc?id=1RBVuPoOFTTSGioAOsUzy76wPSFg3hw5J
To: /content/train.zip
100% 570M/570M [00:10<00:00, 52.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1AIIsrHpftkGEw0-8iKoLLrPoRCNWHq5R
To: /content/test1.zip
100% 284M/284M [00:03<00:00, 77.4MB/s]


In [ ]:
import os

train_root = "./train"
train_files = sorted(os.listdir(train_root))
print(len(train_files))

25000


# 2. Torchvision + PIL vs. Albumentations + OpenCV

In [ ]:
!pip install -U albumentations

     |████████████████████████████████| 102 kB 5.5 MB/s 
     |████████████████████████████████| 37.1 MB 58.1 MB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


## 2.1 Transforms

In [ ]:
from PIL import Image
import torchvision.transforms as transforms

transform_torchvision = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

file_path = os.path.join(train_root, train_files[0])
image = Image.open(file_path)
image_transformed = transform_torchvision(image)
print(image_transformed)
print(image_transformed.max(), image_transformed.min())

tensor([[[ 0.5922,  0.5922,  0.6000,  ...,  0.8824,  0.8745,  0.8667],
         [ 0.5922,  0.5922,  0.6000,  ...,  0.8902,  0.8824,  0.8667],
         [ 0.5922,  0.5922,  0.6000,  ...,  0.8902,  0.8824,  0.8745],
         ...,
         [ 0.2000,  0.2000,  0.2000,  ..., -0.9843, -0.9843, -0.9843],
         [ 0.1922,  0.1922,  0.1922,  ..., -0.9843, -0.9843, -0.9843],
         [ 0.1843,  0.1843,  0.1843,  ..., -0.9922, -0.9922, -0.9922]],

        [[ 0.2863,  0.2863,  0.2941,  ...,  0.5765,  0.5686,  0.5608],
         [ 0.2863,  0.2863,  0.2941,  ...,  0.5843,  0.5765,  0.5608],
         [ 0.2863,  0.2863,  0.2941,  ...,  0.5843,  0.5765,  0.5686],
         ...,
         [-0.0431, -0.0431, -0.0431,  ..., -0.9843, -0.9843, -0.9843],
         [-0.0510, -0.0510, -0.0510,  ..., -0.9843, -0.9843, -0.9843],
         [-0.0588, -0.0588, -0.0588,  ..., -0.9922, -0.9922, -0.9922]],

        [[-0.3176, -0.3176, -0.3098,  ..., -0.0431, -0.0510, -0.0588],
         [-0.3176, -0.3176, -0.3098,  ..., -0

In [ ]:
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

transform_albumentations = A.Compose([
    A.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ToTensorV2(),
])

file_path = os.path.join(train_root, train_files[0])
image = cv2.imread(file_path, cv2.IMREAD_COLOR)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_transformed = transform_albumentations(image=image)
image_transformed = image_transformed["image"]
print(image_transformed)
print(image_transformed.max(), image_transformed.min())

tensor([[[ 0.5922,  0.5922,  0.6000,  ...,  0.8824,  0.8745,  0.8667],
         [ 0.5922,  0.5922,  0.6000,  ...,  0.8902,  0.8824,  0.8667],
         [ 0.5922,  0.5922,  0.6000,  ...,  0.8902,  0.8824,  0.8745],
         ...,
         [ 0.2000,  0.2000,  0.2000,  ..., -0.9843, -0.9843, -0.9843],
         [ 0.1922,  0.1922,  0.1922,  ..., -0.9843, -0.9843, -0.9843],
         [ 0.1843,  0.1843,  0.1843,  ..., -0.9922, -0.9922, -0.9922]],

        [[ 0.2863,  0.2863,  0.2941,  ...,  0.5765,  0.5686,  0.5608],
         [ 0.2863,  0.2863,  0.2941,  ...,  0.5843,  0.5765,  0.5608],
         [ 0.2863,  0.2863,  0.2941,  ...,  0.5843,  0.5765,  0.5686],
         ...,
         [-0.0431, -0.0431, -0.0431,  ..., -0.9843, -0.9843, -0.9843],
         [-0.0510, -0.0510, -0.0510,  ..., -0.9843, -0.9843, -0.9843],
         [-0.0588, -0.0588, -0.0588,  ..., -0.9922, -0.9922, -0.9922]],

        [[-0.3176, -0.3176, -0.3098,  ..., -0.0431, -0.0510, -0.0588],
         [-0.3176, -0.3176, -0.3098,  ..., -0

## 2.2 Speed

In [ ]:
import time

train_file_path_list = [os.path.join(train_root, f) for f in train_files]

In [ ]:
start = time.time()

transform_torchvision = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

for file_path in train_file_path_list:
    image = Image.open(file_path)
    transform_torchvision(image)

print(f"Elapsed time: {time.time() - start}s")

Elapsed time: 109.78422498703003s


In [ ]:
start = time.time()

transform_albumentations = A.Compose([
    A.Resize(256, 256),
    A.RandomCrop(224, 224),
    A.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ToTensorV2(),
])

for file_path in train_file_path_list:
    image = cv2.imread(file_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    transform_albumentations(image=image)

print(f"Elapsed time: {time.time() - start}s")

Elapsed time: 76.18698382377625s
